In [1]:
import ujson
import numpy 
import datetime

# Load Data

In [2]:
summaries = {}
ctr = 0
for line in open('user_project_summaries.json'): # lazy iteration because the file is large
    print ctr,
    ctr+=1
    summaries.update(ujson.loads(line))


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46


## Language

Create Language Data Set and Initialize Language Codes

In [3]:
'''We are using all the data from the user from whom we we have previously 
classfied what their primary langauge is '''

isocodes = ujson.load(open('isocodes.json'))  # mapping from  iso code to language name
user_langs = ujson.load(open('user_inferredlangs.json'))  # mapping from userid to inferred language
#remove Latin because it's not a reliable inference
#TODO for later: get next-best language besides Latin in the inference function
# remove uncommon languages
lang_counts = {}
for user in user_langs:
    lang = user_langs[user]
    lang_counts[lang] = lang_counts.get(lang, 0) + 1
        
user_langs = {user: lang for user, lang in user_langs.items() if lang!='la' and lang_counts[lang]>=500}

langset = sorted(list(set(user_langs.values())))
lang_num_map = {lang: i for i, lang in enumerate(langset)} #key is langauge isocode, value is number 
print 'lang data created with', len(user_langs), 'users and', len(langset), 'languages'

lang data created with 40875 users and 6 languages


## Time

In [4]:
# time utilities
def convert_time(el):
    """Human readable time"""
    el = int(str(el)[:10])
    return datetime.date.fromtimestamp(el)

def timediff(t1, t2):
    """difference between times (millisec precision) as days"""
    return (t1-t2)/(86400.*1000)

In [5]:
earliest = 1e100
latest = 0

ctr = 0
for user in summaries:
    for project in summaries[user]:
        ctime = summaries[user][project]["**created"]
        if ctime < earliest:
            earliest = ctime
        if ctime > latest:
            latest = ctime
            
    ctr+=1
    if ctr%1000==0:
        print ctr/1000,
            
print
print 'Earliest:', convert_time(earliest) 
print 'Latest:', convert_time(latest)
MAXDUR = 300
MINDUR = 150
# get subset of data from "old" users whose earliest creation date is more than MAXDUR days before the end of the dataset
# (giving them a good chance to stay on for more than MINDUR days),
# AND have activity within MAXDUR days if they are active beyond MINDUR (to be fair to recent users)

old_users = set() 

ctr = 0
for user in summaries:
    ctimes = [project["**created"] for project in summaries[user].values()]
    if ctimes==[]:
        print user, 'has no projects'
        continue
    start_time = min(ctimes)
    if timediff(latest, start_time)>MAXDUR:  # first restrict to old users
        # creation times since start date of user
        ctimes_from_start = [timediff(project["**created"], start_time) for project in summaries[user].values()]  
        # projects created at least MINDUR days after start
        post_mindur = [ctime for ctime in ctimes_from_start if ctime>MINDUR]
        # consider users who either were not active after MINDUR days, or had activity between MINDUR and MAXDUR days
        if post_mindur==[] or len(filter(lambda length: length<=MAXDUR, post_mindur))>0:
            old_users.add(user)
            
    ctr+=1
    if ctr%1000==0:
        print ctr/1000,
        
print
print 'Filtered to', len(old_users), 'users from', len(summaries)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
Earliest: 2013-03-27
Latest: 2016-03-10
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 00999 has no projects

Filtered to 25346 users from 46320


In [151]:
def createKeywordDict(projects): 
    '''creates a dictionary of all of the classes a user used in their projects and how many times each class was used'''
    numScreens = getScreenNames(projects)
    keywordDict = {}
    i = 0
    while i < len(projects):
        for screenName in numScreens[i]:
            if screenName in projects[i].keys():
                if  not isinstance(projects[i][screenName]['Components'], unicode):
#                    if 'Number of Components' in projects[i][screenName]['Components'].keys():
                    if not isinstance(projects[i][screenName]['Components']['Type and Frequency'],unicode):
                        for key in projects[i][screenName]['Components']['Type and Frequency']:
                            if key in keywordDict:
                                keywordDict[key] = keywordDict[key] + projects[i][screenName]['Components']['Type and Frequency'][key]
                            else: 
                                keywordDict[key] = projects[i][screenName]['Components']['Type and Frequency'][key]
        i+=1
    
    return keywordDict 

In [43]:
def projectKeywords(project): 
    '''the same as createKeywordDict, but for one project'''
    keywordDict = {}

    screenNames = [key for key in project.keys() if '*' not in key]
    
    for screenName in screenNames:
        if screenName in project.keys():
            if  not isinstance(project[screenName]['Components'], unicode):
#                    if 'Number of Components' in projects[i][screenName]['Components'].keys():
                if not isinstance(project[screenName]['Components']['Type and Frequency'],unicode):
                    for key in project[screenName]['Components']['Type and Frequency']:
                        if key in keywordDict:
                            keywordDict[key] = keywordDict[key] + project[screenName]['Components']['Type and Frequency'][key]
                        else: 
                            keywordDict[key] = project[screenName]['Components']['Type and Frequency'][key]
    
    return keywordDict 

In [152]:
# for user in first_ten[0:1]: 
#     projects= getProjects(user, "LANGUAGE")
#     print createKeywordDict(projects)

{u'Sound': 2, u'HorizontalArrangement': 24, u'Canvas': 3, u'TextToSpeech': 1, u'AccelerometerSensor': 1, u'VerticalArrangement': 3, u'Button': 5, u'Clock': 2, u'TinyDB': 3, u'WebViewer': 1, u'Label': 17, u'Spinner': 1, u'Slider': 1, u'Camera': 1, u'TableArrangement': 2, u'ListView': 1, u'Image': 7, u'Notifier': 1, u'TextBox': 3}


In [234]:
def blockClassFrequency(projects): 
    '''a dictionary that contains the number of projects
    a user used each class of blocks in that was used by the user at least once'''
    freqDict = {} 
    numProjects = len(projects)
    for project in projects: 
        projectDict = projectKeywords(project)
        keys = projectDict.keys()
        for key in keys: 
            if key not in freqDict.keys(): 
                freqDict[key] = 1
            else: 
                freqDict[key] +=1
    finalKeys = freqDict.keys() 
    for key in finalKeys: 
        freqDict[key] = float(freqDict[key])
    return freqDict

In [237]:
for user in first_ten: 
    print blockClassFrequency(getProjects(user,"LANGUAGE"))

{u'Sound': 2.0, u'HorizontalArrangement': 12.0, u'Canvas': 3.0, u'TextToSpeech': 1.0, u'AccelerometerSensor': 1.0, u'VerticalArrangement': 3.0, u'Button': 5.0, u'Clock': 2.0, u'TinyDB': 2.0, u'WebViewer': 1.0, u'Label': 9.0, u'Spinner': 1.0, u'Slider': 1.0, u'Camera': 1.0, u'TableArrangement': 2.0, u'ListView': 1.0, u'Image': 4.0, u'Notifier': 1.0, u'TextBox': 2.0}
{u'Sound': 12.0, u'HorizontalArrangement': 18.0, u'Canvas': 18.0, u'CheckBox': 1.0, u'TextToSpeech': 6.0, u'AccelerometerSensor': 1.0, u'VerticalArrangement': 2.0, u'Texting': 4.0, u'Button': 24.0, u'Clock': 13.0, u'TinyDB': 3.0, u'WebViewer': 3.0, u'Label': 24.0, u'Player': 5.0, u'Camera': 2.0, u'Slider': 1.0, u'TableArrangement': 7.0, u'LocationSensor': 3.0, u'Image': 3.0, u'Notifier': 1.0, u'TextBox': 4.0}
{u'Sound': 4.0, u'VerticalArrangement': 1.0, u'Canvas': 7.0, u'ActivityStarter': 1.0, u'ListPicker': 1.0, u'AccelerometerSensor': 2.0, u'HorizontalArrangement': 6.0, u'SpeechRecognizer': 2.0, u'Button': 10.0, u'Clock': 

In [159]:
def projectFunctions(project):
    '''returns how many times a user used a function a particular in a project'''
    freqDict = {}
    screenNames = [key for key in project.keys() if '*' not in key]
    i = 0
    while i < len(projects):
        for screenName in screenNames:
            if screenName in project.keys():
                if  not isinstance(project[screenName]['Blocks'], unicode):
                    if  not isinstance(project[screenName]['Blocks']['Active Blocks'],unicode):
                        if project[screenName]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                            all_funcs =  project[screenName]['Blocks']['Active Blocks']['Types'].keys()                                               
                            for func in all_funcs:
                                sfunc = []
                                if "_" in func: 
                                    sfunc = func.split('_')
                                if "." in func:
                                    sfunc = func.split('.')
                                if type(sfunc) == list and len(sfunc) > 1:
                                    key = sfunc[1]
                                    if func[1] not in freqDict.keys(): 
                                         freqDict[key] =  project[screenName]['Blocks']['Active Blocks']['Types'][func]
                                    else: 
                                        freqDict[key] += project[screenName]['Blocks']['Active Blocks']['Types'][func] 
        i+=1
                            
    return freqDict

In [229]:
def blockFuncFrequency(projects):
    '''a dictionary that contains the number of projects
    a user used each function. each function must have been used by the user at least once'''
    freqDict = {} 
    numProjects = len(projects)
    for project in projects: 
        projectDict = projectFunctions(project)
        keys = projectDict.keys()
        for key in keys: 
            if key not in freqDict.keys(): 
                freqDict[key] = 1
            else: 
                freqDict[key] +=1
    finalKeys = freqDict.keys() 
    for key in finalKeys: 
        freqDict[key] = float(freqDict[key])
    return freqDict

In [127]:
def functionDict(projects):
    '''returns a dictionary how many times a user used each function (that was used at least once) throughout her projects. function equivalent to createKeywordDict'''
    freqDict = {}
    screenNames = getScreenNames(projects)
    i = 0
    while i < len(projects):
        for screenName in screenNames[i]:
            if screenName in project.keys():
                if  not isinstance(projects[i][screenName]['Blocks'], unicode):
                    if  not isinstance(projects[i][screenName]['Blocks']['Active Blocks'],unicode):
                        if projects[i][screenName]['Blocks']['Active Blocks'] != 'NO ACTIVE BLOCKS':
                            all_funcs =  projects[i][screenName]['Blocks']['Active Blocks']['Types'].keys()                                               
                            for func in all_funcs:
                                sfunc = []
                                if "_" in func: 
                                    sfunc = func.split('_')
                                if "." in func:
                                    sfunc = func.split('.')
                                if type(sfunc) == list and len(sfunc) > 1:
                                    key = sfunc[1]
                                    if func[1] not in freqDict.keys(): 
                                         freqDict[key] =  projects[i][screenName]['Blocks']['Active Blocks']['Types'][func]
                                    else: 
                                        freqDict[key] += projects[i][screenName]['Blocks']['Active Blocks']['Types'][func] 
        i+=1
                            
    return freqDict

In [202]:
'''this code compiles the frequency of all classes among all users. The frequency dictionary of all classes is allKeysD. 
commonList holds a list of all classes that are used more than 15 times across all users. rareList holds a list of all class that are used fewer than 15 times.'''
allKeys = set()
allKeysD = {}
i=0 
for user in all_users: 
    keywords = createKeywordDict(getProjects(user, "LANGUAGE")).keys()
    for key in keywords: 
        if key not in allKeys: 
            allKeys.add(key)
            allKeysD[key] = 1
        else: 
            allKeysD[key] +=1

    if i % 1000 == 0: 
        print i
    i+=1 
print allKeysD, '\n\n'


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
{u'Web': 14217, u'Canvas': 41014, u'FtcIrSeekerSensor': 6, u'BluetoothServer': 4395, u'ActivityStarter': 24199, u'Texting': 16237, u'SoundRecorder': 5954, u'HorizontalArrangement': 45384, u'Sharing': 6604, u'FloatingButton': 1, u'Sound': 39047, u'LinkedData': 6, u'SpeechRecognizer': 15027, u'mrSoundMeter': 6, u'FtcOpMode': 9, u'SensorDB': 1, u'NxtDrive': 1787, u'LinkedDataForm': 1, u'mrPanel': 1, u'Voting': 9, u'mrButton': 2, u'ProximitySensor': 2090, u'InAppBilling': 1, u'Pedometer': 14, u'NxtColorSensor': 576, u'LightSensor': 3, u'NearField': 2836, u'YandexTranslate': 3262, u'mrSoundRecorder': 1, u'TableArrangement': 31978, u'GalleryViewer': 3, u'CheckBox': 13938, u'Clock': 40555, u'FtcRobotController': 9, u'Label': 456

NameError: name 'K' is not defined

In [211]:
commonList =[]
rareList = []
for k in allKeys: 
    if allKeysD[k] > 15: 
        commonList.append((k, allKeysD[k]))
    else:
        rareList.append((k, allKeysD[k])) 


In [212]:
print sorted(commonList, key= lambda x: x[1])

[(u'AdMob', 26), (u'GyroscopeSensor', 161), (u'FirebaseDB', 201), (u'NxtSoundSensor', 431), (u'NxtLightSensor', 482), (u'NxtColorSensor', 576), (u'NxtTouchSensor', 599), (u'NxtUltrasonicSensor', 950), (u'EmailPicker', 1073), (u'NxtDirectCommands', 1101), (u'NxtDrive', 1787), (u'Twitter', 1932), (u'ProximitySensor', 2090), (u'ContactPicker', 2179), (u'NearField', 2836), (u'YandexTranslate', 3262), (u'TimePicker', 3537), (u'BluetoothServer', 4395), (u'PhoneNumberPicker', 4510), (u'Camcorder', 4584), (u'DatePicker', 4968), (u'ImagePicker', 5119), (u'VideoPlayer', 5794), (u'SoundRecorder', 5954), (u'Spinner', 6041), (u'FusiontablesControl', 6049), (u'Sharing', 6604), (u'PasswordTextBox', 7032), (u'BarcodeScanner', 7252), (u'File', 7405), (u'BluetoothClient', 8726), (u'ListView', 9355), (u'PhoneCall', 10300), (u'TinyWebDB', 11423), (u'CheckBox', 13938), (u'Web', 14217), (u'SpeechRecognizer', 15027), (u'Slider', 15238), (u'Texting', 16237), (u'OrientationSensor', 18248), (u'WebViewer', 20492

In [213]:
print sorted(rareList, key= lambda x: x[1])

[(u'FloatingButton', 1), (u'SensorDB', 1), (u'LinkedDataForm', 1), (u'mrPanel', 1), (u'InAppBilling', 1), (u'mrSoundRecorder', 1), (u'VSVArrangement', 1), (u'MIOIOUltrasonidos', 1), (u'AdAmazonInterstitial', 1), (u'GoogleDrive', 1), (u'MobFoxInterstitial', 1), (u'Sharer', 1), (u'mrSlider', 1), (u'MMedia', 1), (u'MMediaInterstitial', 1), (u'mrCanvas', 1), (u'iSENSE', 1), (u'mrSound', 1), (u'XYChart', 1), (u'Dropbox', 1), (u'mrSphero', 1), (u'mrPlayer', 1), (u'DropBox', 1), (u'UdooArduino', 1), (u'mrButton', 2), (u'PhoneStatus', 2), (u'DigitalRead', 2), (u'UsbAccessory', 2), (u'Chronometer', 2), (u'LightSensor', 3), (u'GalleryViewer', 3), (u'BLE', 3), (u'KitchenSink', 3), (u'AdAmazon', 3), (u'LinkedDataListPicker', 3), (u'WICEDSense', 4), (u'AdMobInterstitial', 5), (u'FtcServo', 5), (u'GoogleMap', 5), (u'FtcIrSeekerSensor', 6), (u'LinkedData', 6), (u'mrSoundMeter', 6), (u'FtcGamepad', 6), (u'FtcOpticalDistanceSensor', 6), (u'FtcDcMotor', 6), (u'FtcDeviceInterfaceModule', 6), (u'HSVArrang

All possible classes[u'Web', u'Canvas', u'FtcIrSeekerSensor', u'BluetoothServer', u'ActivityStarter', u'Texting', u'SoundRecorder', u'HorizontalArrangement', u'Sharing', u'FloatingButton', u'Sound', u'LinkedData', u'SpeechRecognizer', u'mrSoundMeter', u'FtcOpMode', u'SensorDB', u'NxtDrive', u'LinkedDataForm', u'mrPanel', u'Voting', u'mrButton', u'ProximitySensor', u'InAppBilling', u'Pedometer', u'NxtColorSensor', u'LightSensor', u'NearField', u'YandexTranslate', u'mrSoundRecorder', u'TableArrangement', u'GalleryViewer', u'CheckBox', u'Clock', u'FtcRobotController', u'Label', u'Camera', u'EmailPicker', u'AdMobInterstitial', u'NxtTouchSensor', u'ContactPicker', u'FusiontablesControl', u'VSVArrangement', u'PhoneNumberPicker', u'Button', u'NxtDirectCommands', u'ListPicker', u'MIOIOUltrasonidos', u'AdAmazonInterstitial', u'PhoneStatus', u'BarcodeScanner', u'DigitalRead', u'BLE', u'FtcServo', u'GoogleDrive', u'Camcorder', u'Notifier', u'TextBox', u'MobFoxInterstitial', u'Image', u'KitchenSink', u'TinyWebDB', u'GameClient', u'UsbAccessory', u'Sharer', u'Spinner', u'BluetoothClient', u'NxtLightSensor', u'LocationSensor', u'TextToSpeech', u'OrientationSensor', u'FtcGamepad', u'mrSlider', u'ListView', u'FtcOpticalDistanceSensor', u'MMedia', u'WebViewer', u'MMediaInterstitial', u'mrCanvas', u'FtcDcMotor', u'GyroscopeSensor', u'AdAmazon', u'Twitter', u'iSENSE', u'FtcDeviceInterfaceModule', u'VideoPlayer', u'AdMob', u'TimePicker', u'HSVArrangement', u'LinkedDataListPicker', u'FtcTouchSensor', u'mrSound', u'DatePicker', u'VerticalArrangement', u'XYChart', u'FirebaseDB', u'NxtUltrasonicSensor', u'SQLite', u'GoogleMap', u'Slider', u'PasswordTextBox', u'ImagePicker', u'Dropbox', u'WICEDSense', u'TinyDB', u'mrSphero', u'GoogleCloudMessaging', u'mrPlayer', u'Chronometer', u'DropBox', u'AccelerometerSensor', u'PhoneCall', u'Player', u'File', u'NxtSoundSensor', u'UdooArduino

In [10]:
all_users = summaries.keys()

In [13]:
first_thousand = all_users[0:1000]
first_ten = all_users[0:10]

In [131]:
# #list of all functions used
# allFunctions = set()
# allFunctionsD = {}
# i=0 
# for user in all_users: 
#     functions = functionDict(getProjects(user, "LANGUAGE")).keys()
#     for funcName in functions
#         if funcName not in allFunctions: 
#             allFunctions.add(funcName)
#             allFunctionsD[funcName] = 1
#         else: 
#             allFunctionsD[funcName] +=1

#     if i % 1000 == 0: 
#         print i
#     i+=1 
# print allFunctionsD, '\n\n'
# commonFList =[]
# rareFList = []
# for k in allFunctionsD: 
#     if allFunctionsD[k] > 15: 
#         commonFList.append(k)
#     else:
#         rareFList.append(k) 


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
{u'GetDistanceInterval': 458, u'getproperty': 14, u'SetLED': 6, u'SetShowFilterBar': 144, u'SetText': 45613, u'SetApiKey': 3729, u'GoHome': 6035, u'yellow': 12999, u'PlayerError': 3, u'ReceiveUnsigned2ByteNumber': 47, u'DrawPoint': 3419, u'ShowAlert': 23487, u'GetRegId': 1, u'ErrorOccurred': 4000, u'GetSpeed': 6514, u'PictureAfterChanges': 6, u'LsRead': 1, u'ReadBatteryLevel': 1, u'SetHint': 4267, u'GetIsConnected': 4127, u'GetA': 5, u'SetTitle': 8712, u'GetStrength': 6, u'GetX': 22804, u'GetY': 23981, u'GetZ': 209, u'SetPressedEventEnabled': 17, u'GetServiceURL': 137, u'digitalRead1': 2, u'SetStopBeforeDisconnect': 5, u'Now': 14992, u'GetAddressesAndNames': 7350, u'TelemetryAddTextData': 6, u'division': 35288, u'SystemTi

In [221]:
for k in allFunctionsD: 
    if allFunctionsD[k] > 15: 
        commonFList.append((k, allFunctionsD[k]))
    else:
        rareFList.append((k, allFunctionsD[k])) 

#print sorted(commonFList, key= lambda y: y[1]), '\n'
#print sorted(rareFList, key= lambda z: z[1])

[(u'GetRegId', 1), (u'LsRead', 1), (u'ReadBatteryLevel', 1), (u'ShowInterstitialAd', 1), (u'GetDpadUp', 1), (u'EnableScroll', 1), (u'GetReleasedEventEnabled', 1), (u'SetImages', 1), (u'methor', 1), (u'GetDpadDown', 1), (u'BatteryLevelUpdated', 1), (u'methos', 1), (u'playerskin', 1), (u'methoi', 1), (u'methol', 1), (u'net', 1), (u'ResultsToSimpleJSON', 1), (u'OnMapClick', 1), (u'GotBallotConfirmation', 1), (u'RsaTextEncrypt', 1), (u'MenuItemSelected', 1), (u'EnableMyLocation', 1), (u'GetPower', 1), (u'AesFileDecrypt', 1), (u'SetInputMode', 1), (u'ScanDevice', 1), (u'GetSelectedDeviceRssi', 1), (u'AdFailedToLoad', 1), (u'GetDeviceList', 1), (u'plane', 1), (u'SetPropertyURI', 1), (u'methogs', 1), (u'GotResponseFromServer', 1), (u'EnableCompass', 1), (u'mode', 1), (u'OnTouchDown', 1), (u'RsaTextDecrypt', 1), (u'iet', 1), (u'OnMapLongClick', 1), (u'StopSoundPlayback', 1), (u'MicOFF', 1), (u'MenuShow', 1), (u'GetRightStickY', 1), (u'GetColorChangedEventEnabled', 1), (u'clock', 1), (u'LightCh

In [225]:
def get_tf(term,project, freqType): 
    numactiveblocks = numActiveBlocks(project) 
    if numactiveblocks == 0: return 0  
    
    if freqType == 'functions': 
        projectFrequency = projectFunctions(project)
    if freqType == 'classes': 
        projectFrequency = projectKeywords(project)
    
    if term in projectFrequency: 
        return projectFrequency[term] / float(numactiveblocks)
    else: return 0

In [232]:
import math
def get_idf(term, projects, freqType): 
    '''idf is log(total number of docs/ number of docs with term in it)'''
    
    if freqType == 'functions': 
        projectFrequency = blockFuncFrequency(projects)
    if freqType == 'classes': 
        projectFrequency = blockClassFrequency(projects)
    
    l = float(numProjects(projects))
    if term not in projectFrequency.keys(): return 0 
    
    f = projectFrequency[term]
   
    if l == 0 or f == 0: return 0
    
    return math.log(l / float(f))

In [227]:
def get_tfidf(term, projects, freqType):
    idf = get_idf(term, projects, freqType)
    all_tf = 0
    for project in projects:
        tf = get_tf(term, project, freqType)
        all_tf += tf
    return (tf/numProjects(projects)) * idf

In [136]:
def numActiveBlocks(project):
    screenNames = [key for key in project.keys() if '*' not in key]
    totalBlocks = 0
    for screenName in screenNames:
        if screenName in project.keys():
            if  not isinstance(project[screenName]['Blocks'], unicode):
                if  not isinstance(project[screenName]['Blocks']['Active Blocks'],unicode):
                    totalBlocks += project[screenName]['Blocks']['Active Blocks']['*Number of Blocks']
    return totalBlocks

In [241]:
for user in first_ten:
    projects = getProjects(user, "LANGUAGE")
    print get_tfidf("HorizontalArrangement", projects, 'classes'), "\n"

0.00204086129148 

0.000754148750514 

0.0 

0.00250469340318 

0.0 

0.0 

0.0 

0.0 

0.0 

0.0 



In [78]:
def getScreenNames(projects): 
    screenNames = []
    for project in projects: 
        projectScreens = [key for key in project.keys() if '*' not in key]
        screenNames.append(projectScreens)
    return screenNames

# Features

In [167]:
from features import *
import features
reload(features)
import sys, os
sys.path.append(os.getcwd())

## Time Features

Get moments from list of project lengths and intervals.

In [243]:
def projectLengthFeatures(projects):
    """moments of project lengths and intervals, as well as number of projects"""
    lengths = getProjectLengths(projects)
    intervals = getProjectIntervals(projects)
    
    #note: feature names are for our reference only
    userDict = {} 
    userDict["mean of lengths"] = numpy.mean(lengths)
    userDict["stddev of lengths"] = numpy.std(lengths)
    userDict["mean of intervals"] = numpy.mean(lengths)
    userDict["stddev of intervals"] = numpy.std(lengths)
    
    userDict["num projects"] = numProjects(projects)
    
    return userDict

Get weekday distibution features.

In [244]:
def dayAnalysisFeatures(projects):
    """number of projects on each day of the week, and the percentage of them on a weekday"""
    byday = numOnDay(projects)

    userDict = {day: byday[i] for i, day in enumerate(["Monday", 
                                                       "Tuesday", 
                                                       "Wednesday", 
                                                       "Thursday", 
                                                       "Friday", 
                                                       "Saturday", 
                                                       "Sunday"])}
    return userDict

Histogram of user's projects in the period.

In [245]:
def decileProjects(projects):
    numbins = 10
    hist = projectsPerUserPeriod(projects, bins=numbins)
    
    userDict = {'decile '+str(i+1): hist[i] for i in range(numbins)}
    return userDict

Here is a function to combine different feature groups.

In [246]:
def combine_featfuncs(funclist):
    def combined(user):
        basedict = funclist[0](user)
        for f in funclist[1:]:
            basedict.update(f(user))
        return basedict
    return combined

## Code Features


In [247]:
def summaryOBlockDecile(projects): 
    hist = decileOrphanBlocks(projects)
    userDict = {'Orphan Decile '+str(i+1): hist[i] for i in range(10)}
    return userDict

In [248]:
def summaryDecileTLBlocks(projects): 
    hist = decileTypesTopLevelBlocks(projects)
    userDict = {'TopLevel Decile '+str(i+1): hist[i] for i in range(10)}
    return userDict

In [249]:
def summaryDecileNumScreens(projects): 
    hist = decileNumScreens(projects)
    userDict = {'NumScreens Decile '+str(i+1): hist[i] for i in range(10)}
    return userDict

In [250]:
def summaryAverages(projects):
    userDict = {} 
    userDict["mean of numScreens"] = numpy.mean(getNumScreens(projects))
    userDict["NB"] = averageNumBlocks(projects)
    userDict["OB"] = getAverageOrphanBlocks(projects)
    
    userDict["TL"] = getAverageTypeTLBlocks(projects)
    userDict["TL2"] = getAverageNumTLBlocks(projects)
    
    userDict["NC"] = averageNumComponents(projects)
    userDict["NTC"] = averageNumTypeComponents(projects)

    userDict["MC"] = aveNumMediaAssets(projects)
    
    userDict["NP"] = averageNumProcedures(projects)
    userDict["NS"] = averageNumStrings(projects) 
    
    #userDict["KW"] = countKeywords(projects)
    
    varList = getAllVariables(projects)
    userDict["local vars"] = varList[0]
    userDict["global vars"] = varList[1]
    
    return userDict

In [251]:
def funcs_tfidf(projects):
    userDict = {}
    userDict["Click"] = get_tfidf("Click", projects, "classes")
    userDict["variable"] = get_tfidf("variable", projects, "classes")
    userDict["SetText"] = get_tfidf("SetText", projects, "classes")
    userDict["number"] = get_tfidf("number", projects, "classes")
    userDict["GetText"] = get_tfidf("GetText", projects, "classes")
    userDict["if"] = get_tfidf("if", projects, "classes")
    userDict["declaration"] = get_tfidf("declaration", projects, "classes")
    userDict["compare"] = get_tfidf("compare", projects, "classes")
    userDict["add"] = get_tfidf("add", projects, "classes")
    userDict["join"] = get_tfidf("join", projects, "classes")

    return userDict

In [252]:
def classes_tfidf(projects): 
    userDict["Button"] = get_tfidf("Button", projects, "functions")
    userDict["Label"] = get_tfidf("Label", projects, "functions")
    userDict["HorizontalArrangement"] = get_tfidf("HorizontalArrangement", projects, "functions")
    userDict["Canvas"] = get_tfidf("Canvas", projects, "functions")
    userDict["TextBox"] = get_tfidf("TextBox", projects, "functions")
    userDict["Clock"] = get_tfidf("Clock", projects, "functions")
    userDict["Sound"] = get_tfidf("Sound", projects, "functions")
    userDict["image"] = get_tfidf("Notifier", projects, "functions")
    userDict["VerticalArrangement"] = get_tfidf("VerticalArrangement", projects, "functions")

    return userDict

# Classification

Get list of projects for each user.

In [253]:
def getAllProjects(userID): #from summaries
    """list of projects sorted by creation times"""
    projectlist = summaries[userID].values()
    return sorted(projectlist,
                  key=lambda project: project['**created']) 

def projectsInMindur(projectlist):
    """filter projects within user's MINDUR duration from earliest"""
    earliest = projectlist[0]['**created']
    filtered_projects = [projectlist[0]]
    for project in projectlist[1:]:
        if timediff(project['**created'], earliest)>MINDUR:
            break
        filtered_projects.append(project)
    return filtered_projects

def userDuration(projectlist):
    """get the duration (difference between last and earliest creation dates)"""
    return timediff(projectlist[-1]['**created'], projectlist[0]['**created'])

def getProjects(userID, task):
    projectlist = getAllProjects(userID)
    if task == 'LANGUAGE': 
        return projectlist
    if task == 'RETENTION':
        return projectsInMindur(projectlist)

Use the appropriate set of users for each task with labels.

In [254]:
def get_user_labels(task):
    if task == 'RETENTION': 
        return {user: int(userDuration(getAllProjects(user))>MINDUR) for user in old_users}
    if task == 'LANGUAGE':
        return {user: lang_num_map[lang] for user, lang in user_langs.items()}

### Featurizing

In [255]:
# featurizer functions
time_combined = combine_featfuncs([projectLengthFeatures, dayAnalysisFeatures, decileProjects,])  
code_combined = combine_featfuncs([summaryDecileNumScreens,summaryOBlockDecile,summaryDecileTLBlocks,summaryAverages, funcs_tfidf, classes_tfidf]) 

In [257]:
from sklearn.feature_extraction import DictVectorizer

all_features = []
time_features = [] 
code_features = []

TASK = 'LANGUAGE'  # toggle between LANGUAGE and RETENTION
userlabels = get_user_labels(TASK)

y = []
ctr = 0
for user in userlabels:
    projects = getProjects(user, TASK)  # this returns projects sorted by creation date
    
    y.append(userlabels[user])
    
    time_features.append(time_combined(projects))
    code_features.append(code_combined(projects))
    
    # merge time and code feature dicts
    all_features_user = time_features[-1].copy()
    all_features_user.update(code_features[-1])
    all_features.append(all_features_user)
    
    ctr+=1
    if ctr%1000==0:
        print '.',

y = numpy.array(y)

timevec = DictVectorizer()
Xtime = timevec.fit_transform(time_features) 

codevec = DictVectorizer()
Xcode = codevec.fit_transform(code_features)

allvec = DictVectorizer()
Xall = allvec.fit_transform(all_features)

ImportError: cannot import name __check_build

Scale and standardize features

In [ ]:
from sklearn.preprocessing import scale
# convert to dense matrices since these are dense anyway
Xtime = scale(Xtime.toarray())
Xcode = scale(Xcode.toarray())
Xall = scale(Xall.toarray())

### Model and Prediction

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.metrics import f1_score 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline 
from plotcnf import plot_confusion_matrix

print 'Chance is', max(numpy.bincount(y))/float(len(y))

foldindices = StratifiedKFold(y)

if TASK=='LANGUAGE':
    labels = map(lambda x:x[0], sorted(lang_num_map.items(), key=lambda x:x[1]))  
    # languages corresponding to class labels as a list
elif TASK=='RETENTION':
    labels = ['left', 'survived']
    
k = 70 #num neighbors for kNN
if TASK == 'RETENTION':
    classifiers = [('knn', KNeighborsClassifier(n_neighbors=k)), 
                   ('logreg', LogisticRegression())]
else:
    classifiers = [('logreg', LogisticRegression(class_weight='balanced', 
                                                             multi_class='multinomial', 
                                                             solver='lbfgs'))]  # no knn because it's too slow

for modelname, model in classifiers:
    for featname, X in [('time', Xtime), ('code', Xcode), ('all', Xall)]:
        print  'Building a', modelname, 'model with', X.shape[1], featname, 'features for', TASK
    
        cvaccs = numpy.zeros(len(foldindices))
        cvf1 = numpy.zeros(len(foldindices))
        for i, (trainidx, testidx) in enumerate(foldindices):
            print 'Fold', i+1
        
            ytrain = y[trainidx]
            ytest = y[testidx]
        
            Xtrain = X[trainidx, :]  
            Xtest = X[testidx, :]
            
            model.fit(Xtrain, ytrain)
            cvaccs[i] = model.score(Xtest, ytest)
        
            predictions = model.predict(Xtest)
            cvf1[i] = f1_score(ytest, predictions)
            
            #print '****', modelname, cvaccs[i]
        # analyze last fold only
        cnf_matrix = confusion_matrix(ytest, predictions)  
        plot_confusion_matrix(cnf_matrix, labels)
        plt.show()
        
        print 'Average accuracy', numpy.mean(cvaccs)
        print 'Average F1 score', numpy.mean(cvf1)